In [1]:
## reinstalling the package
! pwd
! pip uninstall -y cellink
! pip install -q ../../.

/home/lollo/Work/hackathon/code/sc-genetics/docs/notebooks
Found existing installation: cellink 0.0.1
Uninstalling cellink-0.0.1:
  Successfully uninstalled cellink-0.0.1


In [2]:
## importing libraries
import logging
import warnings
import anndata as ad
import numpy as np
import pandas as pd
from pathlib import Path

from cellink.io import read_plink
from cellink import DonorData
from cellink.tl import EQTLData, EQTLPipeline

warnings.filterwarnings("ignore")

logger = logging.getLogger(__name__)

In [3]:
DEBUG = False
target_cell_types = ["CD4 ET", "CD4 NC", "CD8 S100B", "CD8 ET", "B IN", "CD8 NC", "B Mem", "Plasma"]
target_chroms = [
    "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", 
    "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22"
]
cis_window = 1_000_000

In [4]:
## paths
DATA = Path("/home/lollo/Work/hackathon/data/Yazar_OneK1K")
if DEBUG:
    scdata_path = DATA / "debug_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad"
else:
    scdata_path = DATA / "OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz"

dump_dir = "/home/lollo/Work/hackathon/out/dumps"
gene_annotation_path = DATA / "gene_annotation.csv"
plink_path = DATA / f"OneK1K_imputation_post_qc_r2_08/plink"

In [5]:
def load_chrom_gdata(chrom: str):
    plink_file = DATA / f"OneK1K_imputation_post_qc_r2_08/plink/chr{chrom}.dose.filtered.R2_0.8"
    print(plink_file)
    gdata = read_plink(plink_file)
    return gdata

In [6]:
## reading single cell data
scdata = ad.read_h5ad(scdata_path)
## reading annotation data
annotation_df = pd.read_csv(gene_annotation_path).loc[
    :, ["ensembl_gene_id", "start_position", "end_position", "chromosome_name"]
]
annotation_df = annotation_df.loc[annotation_df.ensembl_gene_id.isin(scdata.var_names)]
## merging the scdata.var df with the annotations
merged_df = pd.merge(scdata.var, annotation_df, left_index=True, right_on="ensembl_gene_id")
merged_df = merged_df.rename(
    columns={"ensembl_gene_id": "Geneid", "start_position": "start", "end_position": "end", "chromosome_name": "chrom"}
)
merged_df.index = merged_df["Geneid"]
scdata.var = merged_df

In [7]:
## setting counter of iterations
current_iteration = 1
total_iterations = len(target_chroms)*len(target_cell_types)
## iterating over the target chromosomes
for target_chrom in target_chroms:
    ## retrieving genetics data for current chromosome
    gdata = load_chrom_gdata(target_chrom)
    ## initializing donor data
    data = DonorData(adata=scdata, gdata=gdata, donor_key_in_sc_adata="individual")
    ## initializing eqtl data
    eqtl_data = EQTLData(data, n_sc_comps=15)
    ## running eqtl pipeline and reporting results on all variants
    eqtl = EQTLPipeline(eqtl_data, dump_results=True, file_prefix="eqtl_all", dump_dir=dump_dir)
    ## iterating target cell_types
    for target_cell_type in target_cell_types:
        ## displaying progress message
        print(f"Iteration {current_iteration}/{total_iterations}. Currently processing cell type {target_cell_type} with chromosome {target_chrom}", end="...")
        results_df_all = eqtl.run(target_cell_type, target_chrom, cis_window)
        current_iteration += 1
        print(f"Finished to run the pipeline, dumping a table of shape {results_df.shape} to {dump_dir}")

/home/lollo/Work/hackathon/data/Yazar_OneK1K/OneK1K_imputation_post_qc_r2_08/plink/chr1.dose.filtered.R2_0.8
[2024-11-02 21:32:17,065] INFO:cellink._core.donordata: Keeping 981/1034 donors
[2024-11-02 21:32:17,066] INFO:cellink._core.donordata: Dropping 53/1034 donors from genetic data
[2024-11-02 21:32:17,067] INFO:cellink._core.donordata: Dropping 0/981 donors from single-cell data
Iteration 1/176. Currently processing cell type CD4 ET with chromosome 1...[2024-11-02 21:32:17,901] INFO:cellink._core.donordata: Keeping 981/981 donors
[2024-11-02 21:32:17,902] INFO:cellink._core.donordata: Dropping 0/981 donors from genetic data
[2024-11-02 21:32:17,902] INFO:cellink._core.donordata: Dropping 0/981 donors from single-cell data


  1%|          | 19/1793 [00:40<1:03:50,  2.16s/it]

KeyboardInterrupt: 